# Merge the results from the 3 different methods

### Initializations

In [1]:
import sys
import json
import os
from string import Template
import pandas as pd
from time import sleep
from math import isnan, nan
from rdflib import Graph, URIRef

In [2]:
sys.path.append('..')
from utils import *

## Load and merge the csv files

In [3]:
df_fcu_taxref = pd.read_csv("fcu-taxref/result_fcu_taxref_merge.csv", keep_default_na = False)
df_fcu_geves_taxref = pd.read_csv("fcu-geves-taxref/result3_fcu_geves_taxref_merge.csv", keep_default_na = False)
df_fcu_eppo_taxref = pd.read_csv("fcu-eppo-taxref/result3_fcu_eppo_taxref_merge.csv", keep_default_na = False)

In [4]:
df_merge = pd.concat([df_fcu_taxref, df_fcu_geves_taxref, df_fcu_eppo_taxref], sort=False)

In [5]:
dataframe_preview(df_merge)

== Number of lines: 1118
== Number of unique values:
fcu_concept             337
method                    3
geves_name               79
eppo_scientific_name    327
taxref_ref_full_name    616
taxon                   609
rank                      4
dtype: int64


,fcu_concept,method,geves_name,eppo_scientific_name,taxref_ref_full_name,taxon,rank
0,http://ontology.inrae.fr/frenchcropusage/Ananas,direct,,,"Ananas comosus (L.) Merr., 1917",http://taxref.mnhn.fr/lod/taxon/447782,http://taxref.mnhn.fr/lod/taxrank/Species
1,http://ontology.inrae.fr/frenchcropusage/Anemones,direct,,,Anemone hortensis nothosubsp. fulgens (J.Gay) ...,http://taxref.mnhn.fr/lod/taxon/131396,http://taxref.mnhn.fr/lod/taxrank/SubSpecies
2,http://ontology.inrae.fr/frenchcropusage/Angel...,direct,,,"Dicorynia guianensis Amshoff, 1939",http://taxref.mnhn.fr/lod/taxon/733639,http://taxref.mnhn.fr/lod/taxrank/Species
3,http://ontology.inrae.fr/frenchcropusage/Arach...,direct,,,"Arachis hypogaea L., 1753",http://taxref.mnhn.fr/lod/taxon/611649,http://taxref.mnhn.fr/lod/taxrank/Species
4,http://ontology.inrae.fr/frenchcropusage/Artic...,direct,,,"Cynara cardunculus L., 1753",http://taxref.mnhn.fr/lod/taxon/93783,http://taxref.mnhn.fr/lod/taxrank/Species
5,http://ontology.inrae.fr/frenchcropusage/Asperges,direct,,,"Asparagus albus L., 1753",http://taxref.mnhn.fr/lod/taxon/84265,http://taxref.mnhn.fr/lod/taxrank/Species
6,http://ontology.inrae.fr/frenchcropusage/Attiers,direct,,,"Annona squamosa L., 1753",http://taxref.mnhn.fr/lod/taxon/446901,http://taxref.mnhn.fr/lod/taxrank/Species
7,http://ontology.inrae.fr/frenchcropusage/Auber...,direct,,,"Solanum melongena L., 1753",http://taxref.mnhn.fr/lod/taxon/124075,http://taxref.mnhn.fr/lod/taxrank/Species
8,http://ontology.inrae.fr/frenchcropusage/Avoca...,direct,,,"Persea americana Mill., 1768",http://taxref.mnhn.fr/lod/taxon/447273,http://taxref.mnhn.fr/lod/taxrank/Species
9,http://ontology.inrae.fr/frenchcropusage/Avoines,direct,,,"Avena sativa subsp. sativa L., 1753",http://taxref.mnhn.fr/lod/taxon/132016,http://taxref.mnhn.fr/lod/taxrank/SubSpecies


In [6]:
df_merge.to_excel("result_recall.xlsx", index=False)

## Save alignments as triples

In [7]:
g = Graph()
for index, row in df_merge.iterrows():
    # One triple with a general property
    g.add((URIRef(row['fcu_concept']), URIRef('http://taxref.mnhn.fr/lod/property/candidateAlignment'), URIRef(row['taxon']), ))
    
    # One triple with a property specific to the medhod
    g.add((URIRef(row['fcu_concept']), URIRef('http://taxref.mnhn.fr/lod/property/candidateAlignment' + '_' + row['method']), URIRef(row['taxon']), ))
g.serialize('result_recall.ttl', format='turtle')